In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
# Only run once. Prepares the locations' static features,  
#!python3 '/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data_prep.py'

In [5]:
import pandas as pd
import numpy as np
import json

In [6]:
fips_to_id = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/input/FIPS_ID_mappping.csv', usecols = ['FIPS','ID'])
f2i = {}
f2iK = list(fips_to_id['FIPS'])
f2iV = list(fips_to_id['ID'])
for i in range(0,len(f2iK)):
  f2i[f2iK[i]] = f2iV[i]

In [7]:
############################### CREATING GRAPHS ##############################################################
# Modeled after https://github.com/joey1993/pandemic-forecast/blob/main/pandemic-forecast/code/covid_graph_prep.py, https://github.com/joey1993/pandemic-forecast/blob/main/pandemic-forecast/code/utils.py
import pandas as pd
import json
import csv
import networkx as nx

def pre_graphs():
    # (1) Edges: County-County 'borders' (county1, county2, weight=1)
    # Make bordering county prep file
    c_borders_file = json.load(open('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/countycodes_border.json', 'r'))
    c_borders = {int(k):v for k,v in c_borders_file.items()}
    borders_csv = open('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/counties_borders_csv_weight.csv','w')
    graph_writer = csv.writer(borders_csv, delimiter=',', quotechar='"')
    for c1, c2 in c_borders.items():
        for C2 in c2:
          graph_writer.writerow([c1, C2, 1])
    borders_csv.close()
    c_graph_prep = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/counties_borders_csv_weight.csv', header=None)

    # (2) Edges: State-State 'borders' (state1, state2, weight=1) ####
    # Make bordering state prep file
    state_fips = {}
    state_fipsDF = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/static_state_features.csv', usecols = ['FIPS', 'State'])
    for row in state_fipsDF.iterrows(): state_fips[str(row[1][1])] = int(row[1][0])
    # Make dictionary of state:fips
    s_s_name = json.load(open('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/state-state_bordering.json', 'r')) # Undirected relationships
    # Make directed relationships with fips codes (translate using state_fips' state:id)
    s_s_fips = {} 
    for pair in s_s_name:
        state1 = list(pair.keys())[0]
        state2 = list(pair.values())[0]
        fips1 = state_fips[state1]
        fips2 = state_fips[state2]
        if fips1 not in s_s_fips.keys(): s_s_fips[fips1] = []
        if fips2 not in s_s_fips.keys(): s_s_fips[fips2] = []
        s_s_fips[fips1].append(fips2)
        s_s_fips[fips2].append(fips1)
    s_borders = s_s_fips
    borders_csv = open('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/states_borders_csv_weight.csv','w')
    graph_writer = csv.writer(borders_csv, delimiter=',', quotechar='"')
    for s1, s2 in s_borders.items():
        for S2 in s2:
            graph_writer.writerow([s1, S2, 1])
    borders_csv.close()
    s_graph_prep = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/states_borders_csv_weight.csv', header=None)


    # (3) Edges: County-State 'belongs to' (state, county, weight=1) ####
    # Creating dictionary of stateFIPS:[countyFIPS,...]
    sName_cFIPS = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/state_and_county_static_features.csv', usecols = ['FIPS', 'State'])
    sc_belongs = {}
    for row in sName_cFIPS.iterrows():
        stateName = row[1][1]
        stateFIPS = state_fips[stateName]
        countyFIPS = row[1][0]
        if stateFIPS not in sc_belongs.keys(): sc_belongs[stateFIPS] = []
        sc_belongs[stateFIPS].append(countyFIPS)
    # Creating a dataframe with rows of (stateFIPS, countyFIPS, 1)
    borders_csv = open('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/states_counties_belongs_csv_weight.csv','w')
    graph_writer = csv.writer(borders_csv, delimiter=',', quotechar='"')
    for s, c in sc_belongs.items():
        for C in c:
            graph_writer.writerow([s, C, 1])
    borders_csv.close()
    scb_graph_prep = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/states_counties_belongs_csv_weight.csv', header=None)

    #### Combined Edge Tables
    # 2 Edge Types: County-county 'borders', state-state 'borders' (loc1, loc2, weight=1)
    #sc_graph_prep1 = s_graph_prep
    #sc_graph_prep1 = sc_graph_prep1.append(c_graph_prep)
    # 3 Edge Types: County-county 'borders', state-state 'borders', state-county 'belongs' rows (loc1, loc2, weight=1)
    #sc_graph_prep2 = sc_graph_prep1.append(scb_graph_prep)
    
    return c_borders, s_borders, sc_belongs

def pre_graphs_completedata():
    # (1) Edges: County-County 'borders' (county1, county2, weight=1)
    # Make bordering county prep file
    c_borders_file = json.load(open('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/countycodes_border.json', 'r'))
    c_borders = {int(k):v for k,v in c_borders_file.items()}
    borders_csv = open('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/counties_borders_csv_weight.csv','w')
    graph_writer = csv.writer(borders_csv, delimiter=',', quotechar='"')
    for c1, c2 in c_borders.items():
        for C2 in c2:
            if C2 in f2iK and c1 in f2iK:
                graph_writer.writerow([f2i[c1], f2i[C2], 1])
    borders_csv.close()
    c_graph_prep = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/counties_borders_csv_weight.csv', header=None)

    # (2) Edges: State-State 'borders' (state1, state2, weight=1) ####
    # Make bordering state prep file
    state_fips = {}
    state_fipsDF = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/static_state_features.csv', usecols = ['FIPS', 'State'])
    for row in state_fipsDF.iterrows(): state_fips[str(row[1][1])] = int(row[1][0])
    # Make dictionary of state:fips
    s_s_name = json.load(open('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/state-state_bordering.json', 'r')) # Undirected relationships
    # Make directed relationships with fips codes (translate using state_fips' state:id)
    s_s_fips = {} 
    for pair in s_s_name:
        state1 = list(pair.keys())[0]
        state2 = list(pair.values())[0]
        fips1 = state_fips[state1]
        fips2 = state_fips[state2]
        if fips1 not in s_s_fips.keys(): s_s_fips[fips1] = []
        if fips2 not in s_s_fips.keys(): s_s_fips[fips2] = []
        s_s_fips[fips1].append(fips2)
        s_s_fips[fips2].append(fips1)
    s_borders = s_s_fips
    borders_csv = open('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/states_borders_csv_weight.csv','w')
    graph_writer = csv.writer(borders_csv, delimiter=',', quotechar='"')
    for s1, s2 in s_borders.items():
        for S2 in s2:
            if S2 in f2iK and s1 in f2iK:
                graph_writer.writerow([f2i[s1], f2i[S2], 1])
    borders_csv.close()
    s_graph_prep = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/states_borders_csv_weight.csv', header=None)


    # (3) Edges: County-State 'belongs to' (state, county, weight=1) ####
    # Creating dictionary of stateFIPS:[countyFIPS,...]
    sName_cFIPS = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/state_and_county_static_features.csv', usecols = ['FIPS', 'State'])
    sc_belongs = {}
    for row in sName_cFIPS.iterrows():
        stateName = row[1][1]
        stateFIPS = state_fips[stateName]
        countyFIPS = row[1][0]
        if stateFIPS not in sc_belongs.keys(): sc_belongs[stateFIPS] = []
        sc_belongs[stateFIPS].append(countyFIPS)
    # Creating a dataframe with rows of (stateFIPS, countyFIPS, 1)
    borders_csv = open('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/states_counties_belongs_csv_weight.csv','w')
    graph_writer = csv.writer(borders_csv, delimiter=',', quotechar='"')
    for s, c in sc_belongs.items():
        for C in c:
            if C in f2iK and s in f2iK:
              graph_writer.writerow([f2i[s], f2i[C], 1])
    borders_csv.close()
    scb_graph_prep = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/states_counties_belongs_csv_weight.csv', header=None)


    
    return c_borders, s_borders, sc_belongs


####### Graph Construction Functions
#### OPTION 1:  Graph of bordering counties
def create_tmp_graphs1(date_range):
    c_borders, s_borders, sc_belongs = pre_graphs()
    G = nx.Graph() # Undirected graph
    Gs = []
    #nodes = (len(c_borders))
    edges = []
    for k, v in c_borders.items():
        for V in v:
            edges.append((k,V))
    G.add_edges_from(edges)
    print("Total number of nodes: ", int(G.number_of_nodes()))
    print("Total number of edges: ", int(G.number_of_edges()))
    for date in date_range:
        Gs.append(G)
    return Gs

#### OPTION 2: Graph of bordering counties, bordering states
def create_tmp_graphs2(date_range):
    c_borders, s_borders, sc_belongs = pre_graphs()
    G = nx.Graph() # Directed graph
    Gs = []
    #nodes = len(c_borders) + len(s_borders)
    edges = []
    for k, v in c_borders.items():
        for V in v:
            edges.append((k,V))
    for k, v in s_borders.items():
        for V in v:
            edges.append((k,V))
    G.add_edges_from(edges)
    print("Total number of nodes: ", int(G.number_of_nodes()))
    print("Total number of edges: ", int(G.number_of_edges()))
    for date in date_range:
        Gs.append(G)
    return Gs

#### OPTION 3: Graph of bordering counties, bordering states, county-state edges
def create_tmp_graphs3(date_range):
    c_borders, s_borders, sc_belongs = pre_graphs()
    G = nx.DiGraph() # Directed graph
    Gs = []
    #nodes = len(c_borders) + len(s_borders)
    edges = []
    for c1, c2 in c_borders.items(): # county<->county 'borders'
        for C2 in c2:
            edges.append((c1,C2))
            edges.append((C2,c1))
    for s1, s2 in s_borders.items(): # state<->state 'borders'
        for S2 in s2:
            edges.append((s1,S2))
            edges.append((S2,s1))
    for s, c in sc_belongs.items(): # county->state 'belongs
        for C in c:
            edges.append((C,s))

    G.add_edges_from(edges)
    print("Total number of nodes: ", int(G.number_of_nodes()))
    print("Total number of edges: ", int(G.number_of_edges()))
    for date in date_range:
        Gs.append(G)
    return Gs


#### OPTION 3b: Graph of bordering counties, bordering states, county-state edges, complete data only
def create_tmp_graphs3b(date_range, f2iK, f2i):
    c_borders, s_borders, sc_belongs = pre_graphs_completedata()
    G = nx.DiGraph() # Directed graph
    Gs = []
    #nodes = len(c_borders) + len(s_borders)
    edges = []
    for c1, c2 in c_borders.items(): # county<->county 'borders'
        for C2 in c2:
          if C2 in f2iK and c1 in f2iK:
              edges.append((f2i[c1],f2i[C2]))
              edges.append((f2i[C2],f2i[c1]))
    for s1, s2 in s_borders.items(): # state<->state 'borders'
        for S2 in s2:
            if S2 in f2iK and s1 in f2iK:
              edges.append((f2i[s1],f2i[S2]))
              edges.append((f2i[S2],f2i[s1]))
    for s, c in sc_belongs.items(): # county->state 'belongs
        for C in c:
            if s in f2iK and C in f2iK:
              edges.append((f2i[C],f2i[s]))

    G.add_edges_from(edges)
    print("Total number of nodes: ", int(G.number_of_nodes()))
    print("Total number of edges: ", int(G.number_of_edges()))
    for date in date_range:
        Gs.append(G)
    return Gs

#### OPTION 4b: Graph of bordering counties, bordering states, county-state edges, risk factors, complete data only
def create_tmp_graphs4b(dates, f2iK, f2i):
    c_borders, s_borders, sc_belongs = pre_graphs_completedata()
    Gs = []
    #nodes = len(c_borders) + len(s_borders)
    static_edges = []
    for c1, c2 in c_borders.items(): # county<->county 'borders'
        for C2 in c2:
          if C2 in f2iK and c1 in f2iK:
              static_edges.append((f2i[c1],f2i[C2]))
              static_edges.append((f2i[C2],f2i[c1]))
    for s1, s2 in s_borders.items(): # state<->state 'borders'
        for S2 in s2:
            if S2 in f2iK and s1 in f2iK:
              static_edges.append((f2i[s1],f2i[S2]))
              static_edges.append((f2i[S2],f2i[s1]))
    for s, c in sc_belongs.items(): # county->state 'belongs
        for C in c:
            if s in f2iK and C in f2iK:
              static_edges.append((f2i[C],f2i[s]))
    
    ### Add dynamic edges and nodes (weather risk factors for each date)
    state_fips = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/static_state_features.csv', usecols = ['FIPS']) 
    fips2fips = {}
    for i in range(len(state_fips)):   # Translates the 1-2 digit state FIPS to 4-5 digit FIPS 
      fips2fips[np.int64(str(state_fips.iloc[i,0])[:-3])] = state_fips.iloc[i,0]
    # creates a graph for each date
    #print('Dates', dates)
    for d in dates.values():
      G = nx.DiGraph() # Directed graph
      dyn_edges = []
      try:
        df = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/input/risk_factors/{}.csv'.format(d))
        print(len(df))
        for i in range(len(df)):
          state = df.iloc[i,0]
          risk_factor = df.iloc[i,1]
          S = fips2fips[state]
          R = risk_factor
          if S in f2iK:
            dyn_edges.append((f2i[S], R-1))
            dyn_edges.append((R-1, f2i[S]))
      except:
        print('No risk factors for', d)
      G.add_edges_from(static_edges)
      G.add_edges_from(dyn_edges)      
      print("Total number of nodes: ", int(G.number_of_nodes()))
      print("Total number of edges: ", int(G.number_of_edges()))
      print("Total number of dynamic edges:", len(dyn_edges))
      Gs.append(G)

    return Gs

#### OPTION 5b: Graph of bordering counties, bordering states, county-state edges, risk factors, mobility, complete data only
def create_tmp_graphs5b(dates, f2iK, f2i):
    c_borders, s_borders, sc_belongs = pre_graphs_completedata()
    Gs = []
    #nodes = len(c_borders) + len(s_borders)
    static_edges = []
    for c1, c2 in c_borders.items(): # county<->county 'borders'
        for C2 in c2:
          if C2 in f2iK and c1 in f2iK:
              static_edges.append((f2i[c1],f2i[C2]))
              static_edges.append((f2i[C2],f2i[c1]))
    for s1, s2 in s_borders.items(): # state<->state 'borders'
        for S2 in s2:
            if S2 in f2iK and s1 in f2iK:
              static_edges.append((f2i[s1],f2i[S2]))
              static_edges.append((f2i[S2],f2i[s1]))
    for s, c in sc_belongs.items(): # county->state 'belongs
        for C in c:
            if s in f2iK and C in f2iK:
              static_edges.append((f2i[C],f2i[s], 1))
    
    ### Add dynamic edges and nodes (weather risk factors for each date)
    state_fips = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/static_state_features.csv', usecols = ['FIPS']) 
    fips2fips = {}
    for i in range(len(state_fips)):   # Translates the 1-2 digit state FIPS to 4-5 digit FIPS 
      fips2fips[np.int64(str(state_fips.iloc[i,0])[:-3])] = state_fips.iloc[i,0]
    # creates a graph for each date
    #print('Dates', dates)
    for d in dates.values():
      G = nx.DiGraph() # Directed graph
      dyn_edges = []
      # Add risk factor nodes
      try:
        df = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/input/risk_factors/{}.csv'.format(d))
        print(len(df))
        for i in range(len(df)):
          state = df.iloc[i,0]
          risk_factor = df.iloc[i,1]
          S = fips2fips[state]
          R = risk_factor
          if S in f2iK:
            dyn_edges.append(f2i[S], R-1, weight=1)
            dyn_edges.append(R-1, f2i[S],weight=1)
      except:
        print('No risk factors for', d)
      # Add mobility edges
        try: mobility_df = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/input/{}-aggregation.csv'.format(d[:7]))
        except: mobility_df = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/input/{}-aggregation.csv'.format('2021-10'))
        for j in range(len(mobility_df)):
          fromFIPS = mobility_df.iloc[j,0]
          toFIPS = mobility_df.iloc[j,1]
          people = mobility_df.iloc[j,2]
          G.add_edge(f2i[fromFIPS], f2i[toFIPS], weight=people/100)
      G.add_edges_from(static_edges, weight=1)
      G.add_edges_from(dyn_edges, weight=1)      
      Gs.append(G)

    return Gs

In [8]:

################################################# Features #################################################

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Choose a range of date for which we want to generate the feature matrix 
# The earliest start date allowed is 2021-01-01
start_date = '2021-08-01T00:00:00.000'
end_date = '2021-11-28T00:00:00.000'

# Read in the dynamic and static data files
df_state_static = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/static_state_features.csv')
df_county_static = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/counties_static_features.csv')
df_covid_cases = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/CDC_COVID19_cases_and_vaccine_time_series/covid_cases_by_county.csv')
df_covid_vaccine = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/CDC_COVID19_cases_and_vaccine_time_series/vaccine_status_by_county.csv')

# Delete the unnecessary indexing column
df_state_static = df_state_static.drop(df_state_static.columns[0], axis=1)
df_county_static = df_county_static.drop(df_county_static.columns[0], axis=1)
df_covid_cases = df_covid_cases.drop(df_covid_cases.columns[0], axis = 1)

# For county data, we don't need the column that indicates the state
df_county_static = df_county_static.drop('State', 1)

# Combine the state and county static features together into a single matrix
# The first 51 rows are state-wise features, the rest of the 3142 rows are county-wise features
# Both state and county have the same kind of features
df_state_static = df_state_static.rename({'State': 'location'}, axis='columns') 
df_county_static = df_county_static.rename({'County': 'location'}, axis='columns')
df_state_and_county_static = pd.concat([df_state_static, df_county_static])
df_state_and_county_static = df_state_and_county_static.reset_index(drop = True)
df_state_and_county_static['location'] = df_state_and_county_static['location'].str.lower()
# Create a location name -- FIPS mapper in a dictionary
location_fips_mapping = dict(zip(df_state_and_county_static['location'][0:51], df_state_and_county_static['FIPS'][0:51]))


# For COVID cases data, drop any rows that have missing values for cases_per_100k_7_day_count or percent_test_results_reported
df_covid_cases['cases_per_100k_7_day_count'] = df_covid_cases['cases_per_100k_7_day_count'].replace('suppressed', np.nan)

# Count number of missing data for each COVID variable
print('Number of missing for cases per 100k ', df_covid_cases['cases_per_100k_7_day_count'].isnull().sum())
print('Number of missing for percent_test ', df_covid_cases['percent_test_results_reported'].isnull().sum())

df_covid_cases_no_missing = df_covid_cases.dropna(subset = ['cases_per_100k_7_day_count'])
df_covid_cases_no_missing = df_covid_cases_no_missing.reset_index(drop = True)

# Clean the numbers by getting rid of the comma in the numbers
df_covid_cases_no_missing['cases_per_100k_7_day_count'] = df_covid_cases_no_missing.apply(lambda x: x['cases_per_100k_7_day_count'].replace(',', ''),axis=1)
df_covid_cases_no_missing['cases_per_100k_7_day_count'] = pd.to_numeric(df_covid_cases_no_missing['cases_per_100k_7_day_count'])
#df_covid_cases_no_missing['percent_test_results_reported'] = pd.to_numeric(df_covid_cases_no_missing['percent_test_results_reported'])

# For vaccine data, drop any rows that have missing values for series_complete_pop_pct 
df_covid_vaccine_no_missing = df_covid_vaccine.dropna(subset = ['series_complete_yes'])
df_covid_vaccine_no_missing = df_covid_vaccine_no_missing.reset_index(drop = True)

print('Number of rows with missing values for COVID cases info from March 1 2020 to Nov 2021', len(df_covid_cases) - len(df_covid_cases_no_missing))
print('Number of rows with missing values for COVID vaccine info from March 1 2020 to Nov 2021', len(df_covid_vaccine) - len(df_covid_vaccine_no_missing))

# Reference: https://gist.github.com/rogerallen/1583593
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "Washington DC": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

# Swap the key and value because we want to convert abbreviation to full name
us_abbrev_to_state = dict((v,k) for k,v in us_state_to_abbrev.items())

# From the entire COVID dynamic data, take out the data within a desired date range
# Here sorting does not affect results. It was used for debugging purpose.
def select_data_by_dates(df_input):
  df_sorted = df_input.sort_values(['date', 'state_name'])
  df_sorted['date'] = pd.to_datetime(df_sorted['date'])

  # Incorporate additional past 7 days from the starting day because we need to generate the historical 7-day COVID information later on.
  df_sorted = df_sorted[df_sorted['date'] >= pd.to_datetime(start_date) - timedelta(days = 7)]
  df_sorted = df_sorted[df_sorted['date'] <= pd.to_datetime(end_date)]

  df_sorted = df_sorted.reset_index(drop = True)
  df = df_sorted
  return df

# Rename some columns for vaccine data to be the same as the ones in the covid cases data so that later we can merge vaccine and cases data together
df_covid_vaccine_no_missing = df_covid_vaccine_no_missing.rename(columns= {'date': 'date', 
                                                                           'fips': 'fips_code', 'recip_county': 'county_name', 
                                                                           'recip_state': 'state_name'})

df_cases_selected_by_dates = select_data_by_dates(df_covid_cases_no_missing)
df_vaccine_selected_by_dates = select_data_by_dates(df_covid_vaccine_no_missing)

# Modify the location names of the covid cases dynamic data so that it is consistent with the static data
# Convert the state abbrev to full name for vaccine data
df_vaccine_selected_by_dates['state_name'] = df_vaccine_selected_by_dates['state_name'].map(us_abbrev_to_state)
df_vaccine_selected_by_dates['state_name'] = df_vaccine_selected_by_dates['state_name'].str.lower()
df_vaccine_selected_by_dates['county_name'] = df_vaccine_selected_by_dates['county_name'].str.lower()
# Somehow there are rows with FIPS code be "UNK". Need to drop these rows.
df_vaccine_selected_by_dates = df_vaccine_selected_by_dates[df_vaccine_selected_by_dates['fips_code'] != 'UNK']
df_vaccine_selected_by_dates = df_vaccine_selected_by_dates.reset_index(drop = True)
df_vaccine_selected_by_dates['fips_code'] = pd.to_numeric(df_vaccine_selected_by_dates['fips_code'])

df_cases_selected_by_dates['state_name'] = df_cases_selected_by_dates['state_name'].str.lower()
df_cases_selected_by_dates['county_name'] = df_cases_selected_by_dates['county_name'].str.lower()
df_cases_selected_by_dates['state_name'] = df_cases_selected_by_dates['state_name'].replace('district of columbia', 'washington dc')
df_cases_selected_by_dates['county_name'] = df_cases_selected_by_dates['county_name'].replace('district of columbia', 'washington dc')
df_cases_selected_by_dates['county_name'] = df_cases_selected_by_dates['county_name'].replace('lasalle parish', 'la salle parish')
df_cases_selected_by_dates['county_name'] = df_cases_selected_by_dates['county_name'].replace('doña ana county', 'dona ana county')

# Get rid of locations that don't exist in the static data 
df_vaccine_selected_by_dates = df_vaccine_selected_by_dates[df_vaccine_selected_by_dates['state_name'] != 'american samoa'] 
df_vaccine_selected_by_dates = df_vaccine_selected_by_dates[df_vaccine_selected_by_dates['state_name'] != 'northern mariana islands'] 
df_vaccine_selected_by_dates = df_vaccine_selected_by_dates[df_vaccine_selected_by_dates['state_name'] != 'guam'] 
df_vaccine_selected_by_dates = df_vaccine_selected_by_dates[df_vaccine_selected_by_dates['state_name'] != 'puerto rico'] 
df_vaccine_selected_by_dates = df_vaccine_selected_by_dates[df_vaccine_selected_by_dates['state_name'] != 'u.s. virgin islands'] 


df_cases_selected_by_dates = df_cases_selected_by_dates[df_cases_selected_by_dates['state_name'] != 'puerto rico'] 
df_cases_selected_by_dates = df_cases_selected_by_dates.reset_index(drop = True)

# Store the dates in a list so that later we can loop through each date and create relevant data for each date.
dates_list = df_cases_selected_by_dates['date'].drop_duplicates()
dates_list = dates_list.reset_index(drop = True)

def create_cleaned_df_for_dynamic_features(dates_list, df_input, variable_name):
  print('For variable ', variable_name)
  results_across_days = []
  for date_index in range(len(dates_list)):
    # Process each day separately 
    date = dates_list[date_index]
    groups_by_date = df_input.groupby('date')
    print('Processing ', date)

    # Find the rows/locations corresponding to the same date
    df_data_with_same_date = groups_by_date.get_group(date)
    df_data_with_same_date = df_data_with_same_date.reset_index(drop = True)

    # Calculate state-level COVID numbers by aggregating across the counties in that state
    # Within the rows with the same date, find the ones with the same US state name
    groups_by_state_indices = df_data_with_same_date.groupby('state_name').indices
    for key, value in groups_by_state_indices.items():
      state_name = key
      # For each state, get its COVID data by summing over all its counties 
      FIPS_of_location = location_fips_mapping[state_name]
      state_level_result = sum(df_data_with_same_date[variable_name][value])
      results_across_days.append((date, state_name, FIPS_of_location, state_level_result))

    # After getting the state-level data, we simply save the county level COVID data 
    for county_index in range(len(df_data_with_same_date)): 
      county_name = df_data_with_same_date['county_name'][county_index]
      county_FIPS = df_data_with_same_date['fips_code'][county_index]
      county_level_result = df_data_with_same_date[variable_name][county_index]
      results_across_days.append((date, county_name, county_FIPS, county_level_result))

  # Save as a dataframe
  df_results = pd.DataFrame(results_across_days, columns = ['date', 'location', 'FIPS', variable_name])
  # Convert the date type into datetime so that the dates can be calculated later
  df_results['date'] = pd.to_datetime(df_results['date'])
  print('Number of resulting rows ', len(df_results))
  return df_results

df_cleaned_covid_cases = create_cleaned_df_for_dynamic_features(dates_list, df_cases_selected_by_dates, 'cases_per_100k_7_day_count')
df_cleaned_covid_vaccine = create_cleaned_df_for_dynamic_features(dates_list, df_vaccine_selected_by_dates, 'series_complete_yes')


variable_name = 'cases_per_100k_7_day_count'
grouped_results = df_cleaned_covid_cases.groupby('date')
df_merged_temporal_current = grouped_results.get_group(dates_list[0])
for date_index in range(1, len(dates_list)):
  df_nth_day = grouped_results.get_group(dates_list[date_index])
  # Some preprocessing on column names to make the merging easier
  df_nth_day = df_nth_day.drop('date', 1)
  df_nth_day = df_nth_day.reset_index(drop = True)
  df_nth_day = df_nth_day.rename(columns = {variable_name: str(dates_list[date_index])[0:10]})

  # Merging 
  df_merged_temporal = pd.merge(df_nth_day, df_merged_temporal_current, on = 'FIPS')
  df_merged_temporal = df_merged_temporal.drop('location_y', 1)
  df_merged_temporal = df_merged_temporal.rename(columns = {'location_x': 'location'})
  df_merged_temporal_current = df_merged_temporal
  
df_merged_temporal_current = df_merged_temporal_current.drop('date', 1)
df_merged_temporal_current = df_merged_temporal_current.rename(columns = {'cases_per_100k_7_day_count' : str(dates_list[0])[0:10]})
df_merged_temporal_current = df_merged_temporal_current.sort_values(by = 'FIPS')
df_merged_temporal_current = df_merged_temporal_current.reset_index(drop = True)


# When making the feature matrix, we only use the locations that have complete data
locations_to_keep = df_merged_temporal_current['FIPS'].tolist()
df_cleaned_covid_cases_complete_data = df_cleaned_covid_cases.query('FIPS in @locations_to_keep').copy()
df_cleaned_covid_vaccine_complete_data = df_cleaned_covid_vaccine.query('FIPS in @locations_to_keep').copy()
df_state_and_county_static_complete_data = df_state_and_county_static.query('FIPS in @locations_to_keep').copy()

df_cleaned_covid_cases_complete_data = df_cleaned_covid_cases_complete_data.sort_values(by = ['date', 'FIPS'])
df_cleaned_covid_cases_complete_data = df_cleaned_covid_cases_complete_data.reset_index(drop = True)
df_cleaned_covid_vaccine_complete_data = df_cleaned_covid_vaccine_complete_data.sort_values(by = ['date', 'FIPS'])
df_cleaned_covid_vaccine_complete_data = df_cleaned_covid_vaccine_complete_data.reset_index(drop = True)
df_state_and_county_static_complete_data = df_state_and_county_static_complete_data.sort_values(by = 'FIPS')
df_state_and_county_static_complete_data = df_state_and_county_static_complete_data.reset_index(drop = True)
df_cleaned_covid_cases_vaccine_complete_data = df_cleaned_covid_cases_complete_data
df_cleaned_covid_cases_vaccine_complete_data.insert(3, 'series_complete_yes', df_cleaned_covid_vaccine_complete_data['series_complete_yes'].values)


# Clean the static data
# Delete the static features that have more than 50 locations having missing values for it
df_state_and_county_static_complete_data = df_state_and_county_static_complete_data.dropna(thresh = (len(df_state_and_county_static_complete_data) - 75), axis = 1)

# For other features, use average feature column value to impute the missing values
df_state_and_county_static_complete_data = df_state_and_county_static_complete_data.fillna(df_state_and_county_static_complete_data.mean())

# Check which features are deleted
set(df_state_and_county_static.columns) - set(df_state_and_county_static_complete_data.columns)

# Store the dates in a list so that later we can loop through each date and create feature matrix for each date.
# The starting date is the user-specified starting date, therefore we won't include the historical dates of the starting date.
def make_feature_matrix(df_input):
  dates_list_excluding_historical_dates = df_input['date'].drop_duplicates()[df_input['date'] >= (df_input['date'].min() + timedelta(days = 7))]
  dates_list_excluding_historical_dates = dates_list_excluding_historical_dates.reset_index(drop = True)

  # Create the feature matrix that contains the previous 7 days' covid information plus the static features 
  grouped_results = df_input.groupby('date')
  n_days_before_today = [1, 2, 3, 4, 5, 6, 7]
  features_list = []
  for date_index in range(len(dates_list_excluding_historical_dates)):
    df_merged_static_plus_temporal = df_state_and_county_static_complete_data
    print('Processing ', dates_list_excluding_historical_dates[date_index])

    # Iteratively retrieve the past Nth day's COVID data and merge it into the current dataframe
    for t in range(len(n_days_before_today)):
      # Retrieve the past Nth day's COVID data
      n = n_days_before_today[t]
      historical_date = dates_list_excluding_historical_dates[date_index] - timedelta(days = n)
      df_historical_nth_day = grouped_results.get_group(historical_date)
      # Some preprocessing on column names to make the merging easier
      df_historical_nth_day = df_historical_nth_day.drop('date', 1)
      df_historical_nth_day = df_historical_nth_day.sort_values(by = 'FIPS')
      df_historical_nth_day = df_historical_nth_day.reset_index(drop = True)
      # Calculate the vaccine data by dividing the raw numbers by population in that location.
      df_historical_nth_day['series_complete_yes_normed'] = df_historical_nth_day['series_complete_yes'] / df_state_and_county_static_complete_data['POPESTIMATE']
      df_historical_nth_day = df_historical_nth_day.drop('series_complete_yes', 1)
      df_historical_nth_day = df_historical_nth_day.rename(columns = {'cases_per_100k_7_day_count': 'cases_per_100k_7_day_count_' + str(n) + '_days_before_current_day'})
      df_historical_nth_day = df_historical_nth_day.rename(columns = {'series_complete_yes_normed': 'series_complete_yes_normed_' + str(n) + '_days_before_current_day'})
      

      # Merging 
      df_merged_static_plus_temporal = pd.merge(df_historical_nth_day, df_merged_static_plus_temporal, on = 'FIPS')
      df_merged_static_plus_temporal = df_merged_static_plus_temporal.drop('location_y', 1)
      df_merged_static_plus_temporal = df_merged_static_plus_temporal.rename(columns = {'location_x': 'location'})
    
    print('Shape of feature matrix ', df_merged_static_plus_temporal.shape)
    df_merged_static_plus_temporal_features_only = df_merged_static_plus_temporal.drop(['location', 'FIPS'], 1)

    # Append 10 rows of zeros at the end of feature matrix
    for i in range(10):
      df_merged_static_plus_temporal_features_only.loc[len(df_merged_static_plus_temporal_features_only)] = 0
    features_list.append(df_merged_static_plus_temporal_features_only.values)
    #df_merged_static_plus_temporal.to_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Data/Location_feature_matrix_per_day/' + variable_name + str(dates_list_excluding_historical_dates[date_index])[0:10] +  '_feature_matrix.csv')
  

  # This will only return the dataframe of the most recent day
  return df_merged_static_plus_temporal, features_list
    # Save each day's feature matrix

df_merged_static_plus_temporal_last_day, features_list = make_feature_matrix(df_cleaned_covid_cases_vaccine_complete_data)



Number of missing for cases per 100k  471771
Number of missing for percent_test  170123
Number of rows with missing values for COVID cases info from March 1 2020 to Nov 2021 471771
Number of rows with missing values for COVID vaccine info from March 1 2020 to Nov 2021 0
For variable  cases_per_100k_7_day_count
Processing  2021-07-25 00:00:00
Processing  2021-07-26 00:00:00
Processing  2021-07-27 00:00:00
Processing  2021-07-28 00:00:00
Processing  2021-07-29 00:00:00
Processing  2021-07-30 00:00:00
Processing  2021-07-31 00:00:00
Processing  2021-08-01 00:00:00
Processing  2021-08-02 00:00:00
Processing  2021-08-03 00:00:00
Processing  2021-08-04 00:00:00
Processing  2021-08-05 00:00:00
Processing  2021-08-06 00:00:00
Processing  2021-08-07 00:00:00
Processing  2021-08-08 00:00:00
Processing  2021-08-09 00:00:00
Processing  2021-08-10 00:00:00
Processing  2021-08-11 00:00:00
Processing  2021-08-12 00:00:00
Processing  2021-08-13 00:00:00
Processing  2021-08-14 00:00:00
Processing  2021

In [9]:
# Convert FIPS into a sequential list of integers and use them as IDs
df_merged_temporal_current['ID'] = range(len(df_merged_temporal_current))
df_FIPS_ID_mappping = df_merged_temporal_current[['FIPS', 'ID']]
df_FIPS_ID_mappping.to_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Data/Location_feature_matrix_per_day/FIPS_ID_mappping.csv')
df_merged_temporal_current = df_merged_temporal_current.drop(['location', 'FIPS', 'ID'], 1)

# Generate the list for y
y_list = []
# Reversed to get the earliest date to be appended to the list first
# minus 7 becasue we don't need the 7-day historical dates of the earliest date
for column_index in reversed(range(len(df_merged_temporal_current.columns) - 7)): 
  y_list.append(df_merged_temporal_current.iloc[:, column_index].values)

In [10]:
import os, sys
from google.colab import drive

In [11]:
#!pip install torch-scatter
#!pip install torch-sparse
!pip install torch-geometric

!pip install transformers
!pip install datasets
!pip install seqeval

!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu111.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu111.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu111.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cu111.html
!pip install torch-geometric --user

     |████████████████████████████████| 325 kB 8.2 MB/s 
     |████████████████████████████████| 407 kB 51.1 MB/s 
     |████████████████████████████████| 45 kB 4.5 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.2-py3-none-any.whl size=535570 sha256=d5110ac4897aebe8bd039b9d0b7e0b8551c917bf8fecac00044de12bbbf03ceb
  Stored in directory: /root/.cache/pip/wheels/3f/08/13/2321517088bb2e95bfd0e45033bb9c923189e5b2078e0be4ef
Successfully built torch-geometric
     |████████████████████████████████| 3.1 MB 7.6 MB/s 
     |████████████████████████████████| 895 kB 53.0 MB/s 
     |████████████████████████████████| 596 kB 56.2 MB/s 
     |████████████████████████████████| 3.3 MB 54.2 MB/s 
     |████████████████████████████████| 61 kB 650 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 6.8 MB/s 
     |███████████████████

In [12]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def generate_new_batches(Gs, features, y, idx, graph_window, shift, batch_size, device, test_sample):
    N = len(idx)
  
    adj_lst = list()
    features_lst = list()
    y_lst = list()
    node_lst = list()

    batch_data = []
    for i in range(0, N, batch_size):
        # For the last batch in the date range, add a batch (list of days between i and N, there are less than 8 days between i and N here)
        if i+batch_size >= N:
            batch_data += [[idx[x] for x in range(i, N)]]
        # Add a batch (list of days between i and i+8)
        else:
            batch_data += [[idx[x] for x in range(i, i+batch_size)]]
    # For each batch (i.e. a list of 8 days) in a list of batches e.g., [2, 4, 6, 8, 10, 12, 14, 16]
    for batch in batch_data:
        adj_tmp = list(); features_tmp = list();   y_tmp = list(); num_tmp = list()
        line_idx = 0

        # For each day in the batch
        for curr_day in batch:
            # For each day in the past 7 days prior to the batch
            for day_in_window in range(curr_day-graph_window+1,curr_day+1):
                adj_tmp.append(nx.adjacency_matrix(Gs[day_in_window-1]).toarray())          
                for nodes_row_of_feat in features[day_in_window]: # for row in 1519 rows x 97 col
                    features_tmp.append(nodes_row_of_feat)  # Append features from past 7 days
                line_idx += len(features[day_in_window])
            y_tmp.append(y[curr_day+shift])
        
        adj_tmp = sparse_mx_to_torch_sparse_tensor(sp.block_diag(adj_tmp))    # tensor of past 7 days' graphs (35, 1519, 1519) --> (53165, 53165)         i.e (window*daysinbatch, nodes, nodes) or 35  graphs 
        adj_lst.append(adj_tmp.to(device))                                    # list of tensors of past 7 days' graphs
        features_tmp = torch.FloatTensor(features_tmp)                        # tensor of past 7 days' fatures, (nodes*window*days_in_batch, features)    i.e. (53165, 97) or (7*5)
        features_lst.append(features_tmp.to(device))                          # a list of tensors of the past 7 days' features
        y_tmp = torch.FloatTensor(y_tmp).reshape(-1)                          # a 1D tensor of true values for 'shift' days ahead
        y_lst.append(y_tmp.to(device))                                        # a list of 1D tensors of true values for 'shift' days ahead

    return adj_lst, features_lst, y_lst, 0


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [13]:

from numpy.lib.arraysetops import ediff1d
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.nn import GCNConv,GATConv
import networkx as nx
import numpy as np
import scipy.sparse as sp
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

class DGNN(nn.Module):
    def __init__(self, nfeat, nhid, nout, n_nodes, window, dropout, nhist=7):
        super(DGNN, self).__init__()

        self.window = window
        self.n_nodes = n_nodes
        self.n_state = 1519
        self.nhid = nhid
        self.nfeat = nfeat
        self.nhist = nhist
        self.conv1 = GATConv(nfeat, nhid, concat=False)
        self.conv2 = GATConv(nhid, nhid, concat=False)
        self.bn1 = nn.BatchNorm1d(nhid)
        self.bn2 = nn.BatchNorm1d(nhid)
        self.rnn1 = nn.GRU(2*nhid, nhid, 1)
        self.rnn2 = nn.GRU(nhid, nhid, 1)
        self.fc1 = nn.Linear(2*nhid+window*nhist, nhid)
        self.fc2 = nn.Linear(nhid, nout)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.w1 = Parameter(torch.Tensor(nhid, 1))
        self.w2 = Parameter(torch.Tensor(nhid, 1))
        self.m = nn.Softmax(dim=0)


    def forward(self, adj, x):
        lst = list()
        weight = adj.coalesce().values()
        adj = adj.coalesce().indices()
        skip = x.view(-1,self.window,self.n_nodes,self.nfeat)  
        skip = skip[:,:,:self.n_state,:]
        skip = torch.transpose(skip, 1, 2).reshape(-1,self.window,self.nfeat)
        skip = skip[:,:,:self.nhist]

        x, (edge,att) = self.conv1(x, adj, return_attention_weights=True)
        x = self.dropout(self.bn1(self.relu(x)))
        x2 = x.reshape(-1,self.n_nodes,self.nhid)
        x2 = x2[:,:self.n_state,:]
        x2 = x2.reshape(-1,self.nhid)
        lst.append(x2)
        x = self.dropout(self.bn2(self.relu(self.conv2(x, adj))))
        x2 = x.reshape(-1,self.n_nodes,self.nhid)
        x2 = x2[:,:self.n_state,:]
        x2 = x2.reshape(-1,self.nhid)
        lst.append(x2)

        x = torch.cat(lst, dim=1)
        x = x.view(-1, self.window, self.n_state, x.size(1))
        x = torch.transpose(x, 0, 1)
        x = x.contiguous().view(self.window, -1, x.size(3))
        out1, hn1 = self.rnn1(x)   
        out2, hn2 = self.rnn2(out1)
        x = torch.cat([hn1[0,:,:],hn2[0,:,:]], dim=1)

        skip = skip.reshape(skip.size(0),-1)
        x = torch.cat([x,skip], dim=1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x)).squeeze()
        x = x.view(-1)
        return x, (edge,att)

In [ ]:
# 1,7,14,28 days ahead

######################################  Experiment ###########################################################
#import graph_construction_funcs
import os
import time
import argparse
import networkx as nx
import numpy as np
import scipy.sparse as sp


import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader
from datetime import date, timedelta

from os.path import isfile, join, exists
from math import ceil

import itertools
import pandas as pd
import json

#zfrom model import DGNN

#from utils import generate_new_features, AverageMeter, generate_new_batches
#from utils import 
#from utils import sparse_mx_to_torch_sparse_tensor as to_sparse_tensor
#from models import DGNN
        
import sys

import random
random.seed(2021)
torch.manual_seed(2021)

    
def train(epoch, adj, features, y):
    optimizer.zero_grad()
    output,_ = model(adj, features)
    loss_train = F.mse_loss(output, y)
    loss_train.backward(retain_graph=True)
    optimizer.step()
    return output, loss_train


def test(adj, features, y):    
    output,(edge,att) = model(adj, features)
    loss_test = F.mse_loss(output, y)
    return output, loss_test, (edge,att) 



if __name__ == '__main__':
    LR = 0.001
    batch_size = 8
    window = 7
    ahead = 14
    the_shift = [1,7,14,28]
    start_exp = 15
    early_stop = 100
    epochs = 1000
    graph_window = 7
    dropout = 0.5
    hidden = 64
    hiddens = '64'
    ratio = 0
    #model_name = 'DGNN_'
    # model_name = 'ODE_baseline'
    # model_name = 'DGNN_mobility'
    #model_name = 'DGNN_risk_factors'
    model_name = 'DGNN_mobility_and_risk_factors'

    start_test = 0
    end_test = 119
    sep = 10
    
    device = torch.device("cuda" if torch.cuda.is_available() else torch.device("cpu"))
    task = "case"

    sdate, edate = date(2021, 8, 1), date(2021, 11, 28)   ##### EDIT: CHANGE DATES
    delta = edate - sdate
    dates = [sdate + timedelta(days=i) for i in range(delta.days+1)]
    dates = [str(date) for date in dates]
    dates = {i:d for i,d in enumerate(dates)}

    # Change graphs so they create with the nodes in the graphs
    # Create dictionary of FIPS:ID for the locations with complete data, the data to be used in the graphs
    
    fips_to_id = pd.read_csv('/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/input/FIPS_ID_mappping.csv', usecols = ['FIPS','ID'])
    f2i = {}
    f2iK = list(fips_to_id['FIPS'])
    f2iV = list(fips_to_id['ID'])
    for i in range(0,len(f2iK)):
      f2i[f2iK[i]] = f2iV[i]
        
    #gs_adj = create_tmp_graphs3b(dates, f2iK, f2i)  # (days, nodes)
    gs_adj = create_tmp_graphs4b(dates, f2iK, f2i)  # (days, nodes)
    features = np.asarray(features_list)            # (days, nodes, features)
    y = np.asarray(y_list)                          # (days, nodes) list length 120 for 120 days, has lists of length 1519 for locations 
    n_samples = len(gs_adj)                         # (days)
    nfeat = features_list[0].shape[1]               # (features)

    n_risk_factors = 10
    n_state = 1519
    n_nodes = n_state + n_risk_factors

    fw = open("/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/results_{}_{}.csv".format(task,model_name),"a")
    fw.write("shift,edge,topk,window,hidden,MAE,STD,sMAPE"+"\n")

    begin=time.time()

	#---- predict days ahead , 0-> next day etc.
    for shift in the_shift:
    #for shift in list([int(x)-1 for x in the_shift.split(",")]): #range(0,ahead), [int(shift)-1]
        for hidden in list([int(x) for x in hiddens.split(",")]):

            fw = open("/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/results_{}_{}.csv".format(model_name,"US"),"a")
            fw2 = open("/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/results_{}_{}.csv".format(task,model_name),"a")                         
            fw3 = open("/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/atts_{}_{}_shift{}.csv".format(task,model_name,shift),"a")
            result,result_rel = [],[]
            exp = 0
            
            for test_sample in range(start_exp,n_samples-shift): 
                
                if test_sample < int(start_test) or test_sample > int(end_test): continue
                exp+=1
                print("testing on day #{}".format(test_sample))
                test_begin=time.time()

                if test_sample>=108:
                    idx_train = random.sample(list(range(window, test_sample-sep-10)), 80)
                    idx_train = idx_train + list(range(test_sample-sep-10, test_sample-sep))
                else: 
                    idx_train = list(range(window-1, test_sample-sep))                        

                idx_val = list(range(test_sample-sep,test_sample,2)) 
                idx_train = idx_train+list(range(test_sample-sep+1,test_sample,2))

                adj_train, features_train, y_train, lidx_train = generate_new_batches(gs_adj, features, y, idx_train, graph_window, shift, batch_size,device,test_sample)
                adj_val, features_val, y_val, lidx_val = generate_new_batches(gs_adj, features, y, idx_val, graph_window,  shift,batch_size, device,test_sample)
                adj_test, features_test, y_test, lidx_test = generate_new_batches(gs_adj, features, y, [test_sample], graph_window,shift, batch_size, device,test_sample) 

                n_train_batches = ceil(len(idx_train)/batch_size)
                n_val_batches = 1
                n_test_batches = 1

                #-------------------- Training
                # Model and optimizer
                stop = False#
                while(not stop):#
                    sign=exp%2
                    model_save_name = "/content/gdrive/My Drive/COVID_DGNN_KG/Final Folder (Draft)/data/output/models/model_{}_{}_{}_{}_{}.pth.tar".format(model_name,task,shift,'edge',sign)
                    if os.path.exists(model_save_name) and exp not in [1,2]:
                        checkpoint = torch.load(model_save_name)
                        model.load_state_dict(checkpoint['state_dict'])
                    else:
                        model = DGNN(nfeat=nfeat, nhid=hidden, nout=1, n_nodes=n_nodes, window=graph_window, dropout=dropout).to(device)
                    
                    optimizer = optim.Adam(model.parameters(), lr=LR)
                    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10)

                    ############################# Train ##############################
                    best_val_acc= 1e8
                    val_among_epochs = []
                    train_among_epochs = []
                    stop = False

                    start = time.time()
                    for epoch in range(epochs):    
                        model.train()
                        train_loss = AverageMeter()
                        for batch in range(n_train_batches):
                            output, loss = train(epoch, adj_train[batch], features_train[batch], y_train[batch])#,lidx_train[batch])
                            train_loss.update(loss.data.item(), output.size(0))

                        # Evaluate on validation set
                        model.eval()
                        output, val_loss, _ = test(adj_val[0], features_val[0], y_val[0])#, lidx_val[0])
                        val_loss = int(val_loss.detach().cpu().numpy())
                        val_loss /= (graph_window*n_state*5)  # only for US
                        if(epoch%10==0):
                            minute=round((time.time() - start)/60,2)
                            print("Epoch:", '%03d' % (epoch + 1), "train_loss=", "{:.2f}".format(train_loss.avg),"val_loss=", "{:.2f}".format(val_loss), "time=", "{:.2f}".format(minute))
                        train_among_epochs.append(train_loss.avg)
                        val_among_epochs.append(val_loss)
                        if(epoch<30 and epoch>10):
                            if(len(set([round(val_e) for val_e in val_among_epochs[-20:]])) == 1 ):
                                break
                        if(epoch>early_stop):
                            eps = 0.05 if task != "death" else 0.005
                            if abs(val_loss - np.mean(val_among_epochs[-50:])) <= eps:
                                if val_loss < best_val_acc:
                                    best_val_acc = val_loss
                                    print(model_save_name, "#########")
                                    torch.save({
                                        'state_dict': model.state_dict(),
                                        'optimizer' : optimizer.state_dict(),
                                    }, model_save_name)
                                break
                        if val_loss < best_val_acc:
                            best_val_acc = val_loss
                            torch.save({
                                'state_dict': model.state_dict(),
                                'optimizer' : optimizer.state_dict(),
                            }, model_save_name)
                        stop = True
                        scheduler.step(val_loss)

                ############################### Testing #####################################
                test_loss = AverageMeter()
                checkpoint = torch.load(model_save_name)
                model.load_state_dict(checkpoint['state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer'])
                model.eval()

                output, loss, (edge,att) = test(adj_test[0], features_test[0], y_test[0])#, lidx_test[0])
                o = output.cpu().detach().numpy()   # Predicted
                l = y_test[0].cpu().numpy()         # Actual [casesLocation1,...casesLocation1519]

                error = np.mean(abs(o-abs(l)))                    # MAE: Average of |Predicted - Actual|
                error_rel = np.mean(abs(o-abs(l))/(o+l+0.00001))  # sMAPE: Average of |Predicted - Actual|/(Predicted + Actual)

                # Print results
                result.append(error)
                result_rel.append(error_rel)
                print("test error={:.2f},{:.2f}, ".format(error, error_rel)+"mean test error={:.2f},{:.2f}".format(np.mean(result), np.mean(result_rel)))
                fw2.write("{},{},{},{:.5f},{:.5f},{:.5f},{:.5f}".format(shift,hidden,test_sample,error,error_rel,np.mean(result),np.mean(result_rel))+'\n')
                fw3.write("{}, {}, {}".format(test_sample, edge, att))
            fw.write(str(shift)+","+str('edge')+","+str(ratio)+","+str(window)+","+str(hidden)+",{:.5f}".format(np.mean(result))+",{:.5f}".format(np.std(result))+",{:.5f}".format(np.mean(result_rel))+"\n")
            fw.close()
            fw2.close()
            fw3.close()
            print("Total time cost: {} minutes.".format(round((time.time()-begin)/60, 2)))


198
Total number of nodes:  1528
Total number of edges:  7954
Total number of dynamic edges: 388
198
Total number of nodes:  1528
Total number of edges:  7954
Total number of dynamic edges: 388
198
Total number of nodes:  1528
Total number of edges:  7954
Total number of dynamic edges: 388
198
Total number of nodes:  1528
Total number of edges:  7954
Total number of dynamic edges: 388
198
Total number of nodes:  1528
Total number of edges:  7954
Total number of dynamic edges: 388
198
Total number of nodes:  1528
Total number of edges:  7954
Total number of dynamic edges: 388
198
Total number of nodes:  1528
Total number of edges:  7954
Total number of dynamic edges: 388
198
Total number of nodes:  1528
Total number of edges:  7954
Total number of dynamic edges: 388
198
Total number of nodes:  1529
Total number of edges:  7954
Total number of dynamic edges: 388
198
Total number of nodes:  1529
Total number of edges:  7954
Total number of dynamic edges: 388
198
Total number of nodes:  15

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


Epoch: 001 train_loss= 18005252.00 val_loss= 303.83 time= 0.01
Epoch: 011 train_loss= 5785092.00 val_loss= 90.15 time= 0.04
Epoch: 021 train_loss= 1514032.00 val_loss= 16.66 time= 0.04
Epoch: 031 train_loss= 2519706.00 val_loss= 18.01 time= 0.05
Epoch: 041 train_loss= 1665785.00 val_loss= 16.22 time= 0.06
Epoch: 051 train_loss= 2291201.75 val_loss= 15.80 time= 0.07
Epoch: 061 train_loss= 1857263.38 val_loss= 15.75 time= 0.07
Epoch: 071 train_loss= 2146117.00 val_loss= 15.74 time= 0.08
Epoch: 081 train_loss= 2191081.25 val_loss= 15.74 time= 0.09
Epoch: 091 train_loss= 2817222.75 val_loss= 15.74 time= 0.09
Epoch: 101 train_loss= 2163181.75 val_loss= 15.74 time= 0.10
test error=191.50,0.12, mean test error=191.50,0.12
testing on day #16
Epoch: 001 train_loss= 21107044.00 val_loss= 356.49 time= 0.01
Epoch: 011 train_loss= 7015142.50 val_loss= 113.61 time= 0.03
Epoch: 021 train_loss= 1819945.38 val_loss= 14.05 time= 0.04
Epoch: 031 train_loss= 1828170.75 val_loss= 17.20 time= 0.04
Epoch: 04